### Homework - PyCitySchools
Using python 3.9.12 and edited in VSCode

# PyCityChools - Analysis

Overall, we see that charter schools perform better than District schools on reading and math. This includes both average scores and overall passing percentages.
Additionally, the Charter schools have lower per-student spending than the District schools.

Charter schools score better on both reading and math, with a nine in ten students passing in both subjects while district schools score just barely more than half.
    * bullet
    * bullet

Small and medium schools score better than large schools, but this appears to be associated with type, as the top passing schools are all charter schools and the bottom passing schools are all district schools, and the district schools are larger than the charter schools in our performance report.
    * bullet
    * bullet

We do not see increased spending align with higher passing rates; the inverse appears to be true. Again this appears to be in line with type, as per student budgets in the top 5 charter schools are lower than in the bottom 5 district schools.
    * bullet
    * bullet

In [57]:
# Initial setup: grabbing the data into pandas dataframes
import pandas as pd
import os

# Get the absolute path of the directory containing the notebook
notebook_dir = os.path.abspath('') # On my machine, this returns /Users/galenmittermann/Documents/GitHub/pandas-challenge
#print(notebook_dir)
# Then I set the path to the project directory
projectDir = os.path.dirname(notebook_dir)
# Then I create the file path relative to the project directory
schoolFilePath = os.path.join(projectDir, "PyCitySchools/Resources", "schools_complete.csv")
studentFilePath = os.path.join(projectDir, "PyCitySchools/Resources", "students_complete.csv")
# Now I read the csv file into a dataframe
schdf = pd.read_csv(schoolFilePath)
studf = pd.read_csv(studentFilePath)
# Look at the dataframes to make sure this worked
print(schdf.head())
print(studf.head())

   School ID            school_name      type  size   budget
0          0      Huang High School  District  2917  1910635
1          1   Figueroa High School  District  2949  1884411
2          2    Shelton High School   Charter  1761  1056600
3          3  Hernandez High School  District  4635  3022020
4          4    Griffin High School   Charter  1468   917500
   Student ID       student_name gender grade        school_name  \
0           0       Paul Bradley      M   9th  Huang High School   
1           1       Victor Smith      M  12th  Huang High School   
2           2    Kevin Rodriguez      M  12th  Huang High School   
3           3  Dr. Richard Scott      M  12th  Huang High School   
4           4         Bonnie Ray      F   9th  Huang High School   

   reading_score  math_score  
0             66          79  
1             94          61  
2             90          60  
3             67          58  
4             97          84  


In [58]:
# Number of schools present: length of a list that contains the unique values in the School ID column
nUniqueSchools = len(schdf['School ID'].unique())
print(nUniqueSchools)

15


In [59]:
# Number of students: length of a list that contains unique values in the student_id column
nUniqueStudents = len(studf['Student ID'].unique())
print(nUniqueStudents)

39170


In [60]:
# Total budget is sum of the budget column
totalBudget = schdf['budget'].sum()
print(totalBudget)

24649428


In [61]:
# Average math score in the students is average of of the math_score column
avgMath = studf['math_score'].mean()
print(avgMath)

78.98537145774827


In [62]:
# Average reading score in the students is average of of the reading_score column
avgReading = studf['reading_score'].mean()
print(avgReading)

81.87784018381414


In [63]:
# Passing math is scoring above 70
nPassMath = studf[(studf["math_score"] >= 70)].count()["student_name"]
passMathPercent = nPassMath / (nUniqueStudents) * 100
print(passMathPercent)


74.9808526933878


In [64]:
# Passing reading is scoring above 70
nPassReading = studf[(studf["reading_score"] >= 70)].count()["student_name"]
passReadingPercent = (nPassReading / nUniqueStudents) * 100
print(passReadingPercent)

85.80546336482001


In [65]:
# Passing overall is passing in both math and reading
nPassAll = studf[
    (studf["math_score"] >= 70) & (studf["reading_score"] >= 70)
].count()["student_name"]
passAllRate = (nPassAll /  nUniqueStudents) * 100
print(passAllRate)

65.17232575950983


In [66]:
# I put the district data into a dictionary, then read that dictionary into a dataframe. I have to transpose the dataframe to get the columns right.
distData = {'Total Schools':nUniqueSchools, 'Total Students':nUniqueStudents, 'Total Budget': totalBudget, 'Average Math': avgMath, 'Average Reading': avgReading, '% Passing Math': passMathPercent, '% Passing Reading': passReadingPercent, '% Overall Passing': passAllRate}
distdf = pd.DataFrame.from_dict(distData, orient = 'index', columns = ['District Results'])
distdf = distdf.transpose()
#print(distdf)

# Formatting, per the example code
distdf["Total Students"] = distdf["Total Students"].map("{:,}".format)
distdf["Total Budget"] = distdf["Total Budget"].map("${:,.2f}".format)
print(distdf)

                  Total Schools Total Students    Total Budget  Average Math  \
District Results           15.0       39,170.0  $24,649,428.00     78.985371   

                  Average Reading  % Passing Math  % Passing Reading  \
District Results         81.87784       74.980853          85.805463   

                  % Overall Passing  
District Results          65.172326  


In [67]:
# Calculating budget per student and adding it as a column
schdf['budget per student'] = schdf['budget']/schdf['size']

In [68]:
# Calculating avg math and avg reading scores and adding as a columns
schAvgMath = studf.groupby('school_name')['math_score'].mean().reset_index()
schAvgReading = studf.groupby('school_name')['reading_score'].mean().reset_index()
# Merge  into schdf on the school_name column
schdf = schdf.merge(schAvgMath, on='school_name')
schdf = schdf.merge(schAvgReading, on='school_name')
schdf = schdf.rename(columns = {'math_score':'average math', 'reading_score':'average reading'})

# Print the updated DataFrames to see if we have it right
print(schdf.head())
print(studf.head())

   School ID            school_name      type  size   budget  \
0          0      Huang High School  District  2917  1910635   
1          1   Figueroa High School  District  2949  1884411   
2          2    Shelton High School   Charter  1761  1056600   
3          3  Hernandez High School  District  4635  3022020   
4          4    Griffin High School   Charter  1468   917500   

   budget per student  average math  average reading  
0               655.0     76.629414        81.182722  
1               639.0     76.711767        81.158020  
2               600.0     83.359455        83.725724  
3               652.0     77.289752        80.934412  
4               625.0     83.351499        83.816757  
   Student ID       student_name gender grade        school_name  \
0           0       Paul Bradley      M   9th  Huang High School   
1           1       Victor Smith      M  12th  Huang High School   
2           2    Kevin Rodriguez      M  12th  Huang High School   
3           3

In [69]:
# Calculating the percent of students per school that pass math and reading.
# I take only the students with a score equal to or above 70, and group them by school name in order to get the count of students left (i.e those selected with passing scores)
# And then I divide by the total count of students in that school, in order to get the total percent of students who are passing
passMathPercentSchool = (studf[studf['math_score'] >= 70].groupby('school_name')['student_name'].count() / studf.groupby('school_name')['student_name'].count()) * 100
passReadingPercentSchool = (studf[studf['reading_score'] >= 70].groupby('school_name')['student_name'].count() / studf.groupby('school_name')['student_name'].count()) * 100

# Calculating the percent of students that pass both math and reading. Same approach, but filtering on two columns and not just one
passAllRateSchool = (studf[(studf['math_score'] >= 70)&(studf['reading_score'] >= 70)].groupby('school_name')['student_name'].count() / studf.groupby('school_name')['student_name'].count()) * 100

# Convert those to a DataFrame with a reset index
passMathPercentSchool = pd.DataFrame({'school_name': passMathPercentSchool.index, '% Passing Math': passMathPercentSchool.values})
passReadingPercentSchool = pd.DataFrame({'school_name': passReadingPercentSchool.index, '% Passing Reading': passReadingPercentSchool.values})
passAllRateSchool = pd.DataFrame({'school_name': passAllRateSchool.index, '% Overall Passing': passAllRateSchool.values})

# Merge the passing rate dataframes into schdf on the school_name column
schdf = schdf.merge(passMathPercentSchool, on='school_name')
schdf = schdf.merge(passReadingPercentSchool, on='school_name')
schdf = schdf.merge(passAllRateSchool, on='school_name')

# Print the school dataframe to verify my answers
print(schdf.sort_values(by=['school_name']))


    School ID            school_name      type  size   budget  \
7           7     Bailey High School  District  4976  3124928   
6           6    Cabrera High School   Charter  1858  1081356   
1           1   Figueroa High School  District  2949  1884411   
13         13       Ford High School  District  2739  1763916   
4           4    Griffin High School   Charter  1468   917500   
3           3  Hernandez High School  District  4635  3022020   
8           8     Holden High School   Charter   427   248087   
0           0      Huang High School  District  2917  1910635   
12         12    Johnson High School  District  4761  3094650   
9           9       Pena High School   Charter   962   585858   
11         11  Rodriguez High School  District  3999  2547363   
2           2    Shelton High School   Charter  1761  1056600   
14         14     Thomas High School   Charter  1635  1043130   
5           5     Wilson High School   Charter  2283  1319574   
10         10     Wright 

In [70]:
# Top and bottom performing schools - I sort the dataframe and save off the 5 best and worst
topSchoolsdf = schdf.sort_values('% Overall Passing', ascending=False)
topSchoolsdf = topSchoolsdf.head(5)
bottomSchoolsdf = schdf.sort_values('% Overall Passing', ascending=True)
bottomSchoolsdf=bottomSchoolsdf.head(5)
print(topSchoolsdf)
print(bottomSchoolsdf)

    School ID          school_name     type  size   budget  \
6           6  Cabrera High School  Charter  1858  1081356   
14         14   Thomas High School  Charter  1635  1043130   
4           4  Griffin High School  Charter  1468   917500   
5           5   Wilson High School  Charter  2283  1319574   
9           9     Pena High School  Charter   962   585858   

    budget per student  average math  average reading  % Passing Math  \
6                582.0     83.061895        83.975780       94.133477   
14               638.0     83.418349        83.848930       93.272171   
4                625.0     83.351499        83.816757       93.392371   
5                578.0     83.274201        83.989488       93.867718   
9                609.0     83.839917        84.044699       94.594595   

    % Passing Reading  % Overall Passing  
6           97.039828          91.334769  
14          97.308869          90.948012  
4           97.138965          90.599455  
5           96.5

In [71]:
# Average math and reading scores by grade. This is just like the average scores by school but grouped on a different column.
gradeAvgMath = studf.groupby('grade')['math_score'].mean().reset_index()
gradeAvgReading = studf.groupby('grade')['reading_score'].mean().reset_index()
# Rename the colums to show that these are averages
gradeAvgMath = gradeAvgMath.rename(columns = {'math_score':'average math score'})
gradeAvgReading = gradeAvgReading.rename(columns = {'reading_score':'average reading score'})
print(gradeAvgMath)
print(gradeAvgReading)

  grade  average math score
0  10th           78.941483
1  11th           79.083548
2  12th           78.993164
3   9th           78.935659
  grade  average reading score
0  10th              81.874410
1  11th              81.885714
2  12th              81.819851
3   9th              81.914358


In [72]:
# Math scores by grade
# Group the data by school and grade and calculate the average math score for each group
school_grade_math = studf.groupby(['school_name', 'grade'])['math_score'].mean()

# Pivot the data so that grades are columns and schools are rows
school_grade_math = school_grade_math.reset_index().pivot(index='school_name', columns='grade', values='math_score')

# Sort the columns in ascending order
school_grade_math = school_grade_math[['9th', '10th', '11th', '12th']]

# Print the resulting dataframe
print(school_grade_math)



grade                        9th       10th       11th       12th
school_name                                                      
Bailey High School     77.083676  76.996772  77.515588  76.492218
Cabrera High School    83.094697  83.154506  82.765560  83.277487
Figueroa High School   76.403037  76.539974  76.884344  77.151369
Ford High School       77.361345  77.672316  76.918058  76.179963
Griffin High School    82.044010  84.229064  83.842105  83.356164
Hernandez High School  77.438495  77.337408  77.136029  77.186567
Holden High School     83.787402  83.429825  85.000000  82.855422
Huang High School      77.027251  75.908735  76.446602  77.225641
Johnson High School    77.187857  76.691117  77.491653  76.863248
Pena High School       83.625455  83.372000  84.328125  84.121547
Rodriguez High School  76.859966  76.612500  76.395626  77.690748
Shelton High School    83.420755  82.917411  83.383495  83.778976
Thomas High School     83.590022  83.087886  83.498795  83.497041
Wilson Hig

In [73]:
# In this section I am keeping the sample code provided on the challenge webpage, so the naming conventions change
# Normally I try to use camelCase where possible

spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = schdf.copy()
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(schdf["budget"]/schdf["size"], spending_bins, labels=labels)

print(school_spending_df)

spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["average math"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["average reading"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

    School ID            school_name      type  size   budget  \
0           0      Huang High School  District  2917  1910635   
1           1   Figueroa High School  District  2949  1884411   
2           2    Shelton High School   Charter  1761  1056600   
3           3  Hernandez High School  District  4635  3022020   
4           4    Griffin High School   Charter  1468   917500   
5           5     Wilson High School   Charter  2283  1319574   
6           6    Cabrera High School   Charter  1858  1081356   
7           7     Bailey High School  District  4976  3124928   
8           8     Holden High School   Charter   427   248087   
9           9       Pena High School   Charter   962   585858   
10         10     Wright High School   Charter  1800  1049400   
11         11  Rodriguez High School  District  3999  2547363   
12         12    Johnson High School  District  4761  3094650   
13         13       Ford High School  District  2739  1763916   
14         14     Thomas 

In [74]:
# Assemble into DataFrame
spending_summary = pd.DataFrame({"Average Math Score":spending_math_scores, "Average Reading Score": spending_reading_scores, "% Passing Math": spending_passing_math, "% Passing Reading": spending_passing_reading, "% Overall Passing": overall_passing_spending})

# Display results
print(spending_summary)

                               Average Math Score  Average Reading Score  \
Spending Ranges (Per Student)                                              
<$585                                   83.455399              83.933814   
$585-630                                81.899826              83.155286   
$630-645                                78.518855              81.624473   
$645-680                                76.997210              81.027843   

                               % Passing Math  % Passing Reading  \
Spending Ranges (Per Student)                                      
<$585                               93.460096          96.610877   
$585-630                            87.133538          92.718205   
$630-645                            73.484209          84.391793   
$645-680                            66.164813          81.133951   

                               % Overall Passing  
Spending Ranges (Per Student)                     
<$585                           

In [75]:
# Scores by school size
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Create a copy of the school summary since it has the student size
school_size_df = schdf.copy()
# Use `pd.cut` to categorize spending based on the bins.
school_size_df["Size Range"] = pd.cut(schdf["size"], size_bins, labels=labels)
print(school_size_df)

    School ID            school_name      type  size   budget  \
0           0      Huang High School  District  2917  1910635   
1           1   Figueroa High School  District  2949  1884411   
2           2    Shelton High School   Charter  1761  1056600   
3           3  Hernandez High School  District  4635  3022020   
4           4    Griffin High School   Charter  1468   917500   
5           5     Wilson High School   Charter  2283  1319574   
6           6    Cabrera High School   Charter  1858  1081356   
7           7     Bailey High School  District  4976  3124928   
8           8     Holden High School   Charter   427   248087   
9           9       Pena High School   Charter   962   585858   
10         10     Wright High School   Charter  1800  1049400   
11         11  Rodriguez High School  District  3999  2547363   
12         12    Johnson High School  District  4761  3094650   
13         13       Ford High School  District  2739  1763916   
14         14     Thomas 

In [76]:
# Replicating the spending rage process but for size range
size_math_scores = school_size_df.groupby(["Size Range"]).mean()["average math"]
size_reading_scores = school_size_df.groupby(["Size Range"]).mean()["average reading"]
size_passing_math = school_size_df.groupby(["Size Range"]).mean()["% Passing Math"]
size_passing_reading = school_size_df.groupby(["Size Range"]).mean()["% Passing Reading"]
overall_passing_size = school_size_df.groupby(["Size Range"]).mean()["% Overall Passing"]

# Assemble into DataFrame
size_summary = pd.DataFrame({"Average Math Score":size_math_scores, "Average Reading Score": size_reading_scores, "% Passing Math": size_passing_math, "% Passing Reading": size_passing_reading, "% Overall Passing": overall_passing_size})

# Display results
print(size_summary)

                    Average Math Score  Average Reading Score  % Passing Math  \
Size Range                                                                      
Small (<1000)                83.821598              83.929843       93.550225   
Medium (1000-2000)           83.374684              83.864438       93.599695   
Large (2000-5000)            77.746417              81.344493       69.963361   

                    % Passing Reading  % Overall Passing  
Size Range                                                
Small (<1000)               96.099437          89.883853  
Medium (1000-2000)          96.790680          90.621535  
Large (2000-5000)           82.766634          58.286003  


In [77]:
# Replicating the spending rage process but for school type
school_type_df = schdf.copy()

type_math_scores = school_type_df.groupby(["type"]).mean()["average math"]
type_reading_scores = school_type_df.groupby(["type"]).mean()["average reading"]
type_passing_math = school_type_df.groupby(["type"]).mean()["% Passing Math"]
type_passing_reading = school_type_df.groupby(["type"]).mean()["% Passing Reading"]
overall_passing_type = school_type_df.groupby(["type"]).mean()["% Overall Passing"]

# Assemble into DataFrame
type_summary = pd.DataFrame({"Average Math Score":type_math_scores, "Average Reading Score": type_reading_scores, "% Passing Math": type_passing_math, "% Passing Reading": type_passing_reading, "% Overall Passing": overall_passing_type})

# Display results
print(type_summary)

          Average Math Score  Average Reading Score  % Passing Math  \
type                                                                  
Charter            83.473852              83.896421       93.620830   
District           76.956733              80.966636       66.548453   

          % Passing Reading  % Overall Passing  
type                                            
Charter           96.586489          90.432244  
District          80.799062          53.672208  
